In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import model_from_json


In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)

x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)


In [3]:
def conv_nn():
    nn = Sequential()

    nn.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=x_train.shape[1:])) #first layer, input
    nn.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    nn.add(MaxPooling2D(pool_size=(2,2)))
    nn.add(Dropout(0.25)) #help overfitting

    nn.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    nn.add(MaxPooling2D(pool_size=(2,2)))
    nn.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    nn.add(MaxPooling2D(pool_size=(2,2)))
    nn.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    nn.add(MaxPooling2D(pool_size=(2,2)))
    nn.add(Dropout(0.25))

    nn.add(Flatten()) #1D
    nn.add(Dense(256, activation='relu')) #wants 1D dataset
    nn.add(Dropout(0.5))
    nn.add(Dense(7, activation='softmax')) #softmax for muliclass probablity (7 to be exact)
    nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nn


In [4]:
mood_model = conv_nn()

In [ ]:
mood_model.fit(x_train, y_train, epochs=50, use_multiprocessing=True, validation_data=(x_test, y_test))

In [ ]:
mood_model.evaluate(x_test, y_test, batch_size=64)

In [ ]:
#export model
model_json = mood_model.to_json()
with open("mood_model.json", "w") as json_file:
    json_file.write(model_json)
mood_model.save_weights('mood_model.h5')
print("Saved model to disk")

In [ ]:
#import model
with open('mood_model.json','r') as f:
    json = f.read()
loaded_model = model_from_json(json)

loaded_model.load_weights("mood_model.h5")

loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
loaded_model.evaluate(x_test, y_test, batch_size=64)
